In [ ]:
# Import Librairies

import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Read Json file

with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

print("The Intent Data", data['intents'])
print()
print("Training Sentences: ", training_sentences)
print()
print("Labels: ", training_labels)
print()
print("Responses: ", responses)

The Intent Data [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay', 'Wassup bro', 'A fin', 'Wech kho'], 'responses': ['Hello', 'Hi', 'Hi there', 'Hi there']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Ciao', 'Tchuss', 'Thella'], 'responses': ['See you later', 'Have a nice day', 'Bye! Come back again', 'Auf Wiedersehen']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Thanks for the help', 'la7afdek', 'Itoub'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!", 'Hania a khouya']}, {'tag': 'about', 'patterns': ['Who are you?', 'What are you?', 'Who you are?', 'Chkoun nta'], 'responses': ['I.m NawSEG, your bot assistant', "I'm NawSEG, an A.I bot", 'Je suis ton Pere ... Starwars']}, {'tag': 'name', 'patterns': ['what is your name', 'what should I call you', 'whats your name?'], 'responses': ['You can call me NawSEG.', "I'm NawSEG!", 'Just call me as NawSEG']}, {'tag': 'help'

In [ ]:
# Label Encoder Method is encoding the levels of categorical features into numeric values
# LabelEncoder encode labels with a value between 0 and n_classes-1 where n is the number of distinct labels (here n )

lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

training_labels

array([4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 7, 7, 7, 7, 7, 7, 0, 0,
       0, 0, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])

In [ ]:
# Tokenization process (Vectorising the Data)

vocab_size = 1000 # the size of the vocabulary in the text data
embedding_dim = 16
max_len = 20
oov_token = "<OOV>" # Out Of Vocabulary are words not present in the vocabulary and will be mapped as UNK token

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len) # 'pre' or 'post' (optional, defaults to 'pre'): remove values from sequences larger than maxlen, either at the beginning or at the end of the sequences.

In [ ]:
# Training the Neural Network of our Chatbot with 3 Hidden layers

model = Sequential() # The sequential API allows you to create models layer-by-layer for most problems. It is limited in that it does not allow you to create models that share layers or have multiple inputs or outputs.
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D()) # Pooling. Its function is to progressively reduce the spatial size of the representation to reduce the amount of parameters and computation in the network
model.add(Dense(16, activation='relu')) # Activation function Relu 1
model.add(Dense(16, activation='relu')) # Activation function Relu 2
model.add(Dense(num_classes, activation='softmax')) # Activation function Softmax

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # This loss function is mathematically same as the categorical_crossentropy.

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

Epoch 73/500
2/2 [==============================] - 0s 6ms/step - loss: 1.9125 - accuracy: 0.4762
Epoch 74/500
2/2 [==============================] - 0s 4ms/step - loss: 1.9072 - accuracy: 0.4762
Epoch 75/500
2/2 [==============================] - 0s 4ms/step - loss: 1.9010 - accuracy: 0.4762
Epoch 76/500
2/2 [==============================] - 0s 5ms/step - loss: 1.8950 - accuracy: 0.4762
Epoch 77/500
2/2 [==============================] - 0s 5ms/step - loss: 1.8895 - accuracy: 0.5000
Epoch 78/500
2/2 [==============================] - 0s 4ms/step - loss: 1.8834 - accuracy: 0.5238
Epoch 79/500
2/2 [==============================] - 0s 4ms/step - loss: 1.8771 - accuracy: 0.5238
Epoch 80/500
2/2 [==============================] - 0s 5ms/step - loss: 1.8708 - accuracy: 0.5000
Epoch 81/500
2/2 [==============================] - 0s 5ms/step - loss: 1.8646 - accuracy: 0.5000
Epoch 82/500
2/2 [==============================] - 0s 5ms/step - loss: 1.8582 - accuracy: 0.5000
Epoch 83/500
2/2 [==

2/2 [==============================] - 0s 5ms/step - loss: 1.2219 - accuracy: 0.6190
Epoch 157/500
2/2 [==============================] - 0s 5ms/step - loss: 1.2122 - accuracy: 0.6190
Epoch 158/500
2/2 [==============================] - 0s 5ms/step - loss: 1.2031 - accuracy: 0.6190
Epoch 159/500
2/2 [==============================] - 0s 6ms/step - loss: 1.1926 - accuracy: 0.6190
Epoch 160/500
2/2 [==============================] - 0s 5ms/step - loss: 1.1833 - accuracy: 0.6190
Epoch 161/500
2/2 [==============================] - 0s 5ms/step - loss: 1.1743 - accuracy: 0.6190
Epoch 162/500
2/2 [==============================] - 0s 4ms/step - loss: 1.1658 - accuracy: 0.6190
Epoch 163/500
2/2 [==============================] - 0s 6ms/step - loss: 1.1585 - accuracy: 0.6429
Epoch 164/500
2/2 [==============================] - 0s 6ms/step - loss: 1.1478 - accuracy: 0.6667
Epoch 165/500
2/2 [==============================] - 0s 5ms/step - loss: 1.1405 - accuracy: 0.6905
Epoch 166/500
2/2 [=====

2/2 [==============================] - 0s 5ms/step - loss: 0.3585 - accuracy: 0.9286
Epoch 321/500
2/2 [==============================] - 0s 5ms/step - loss: 0.3559 - accuracy: 0.9286
Epoch 322/500
2/2 [==============================] - 0s 6ms/step - loss: 0.3539 - accuracy: 0.9286
Epoch 323/500
2/2 [==============================] - 0s 5ms/step - loss: 0.3516 - accuracy: 0.9286
Epoch 324/500
2/2 [==============================] - 0s 5ms/step - loss: 0.3492 - accuracy: 0.9286
Epoch 325/500
2/2 [==============================] - 0s 6ms/step - loss: 0.3469 - accuracy: 0.9286
Epoch 326/500
2/2 [==============================] - 0s 7ms/step - loss: 0.3437 - accuracy: 0.9286
Epoch 327/500
2/2 [==============================] - 0s 7ms/step - loss: 0.3412 - accuracy: 0.9286
Epoch 328/500
2/2 [==============================] - 0s 5ms/step - loss: 0.3393 - accuracy: 0.9286
Epoch 329/500
2/2 [==============================] - 0s 5ms/step - loss: 0.3364 - accuracy: 0.9286
Epoch 330/500
2/2 [=====

2/2 [==============================] - 0s 6ms/step - loss: 0.0753 - accuracy: 1.0000
Epoch 485/500
2/2 [==============================] - 0s 5ms/step - loss: 0.0745 - accuracy: 1.0000
Epoch 486/500
2/2 [==============================] - 0s 5ms/step - loss: 0.0737 - accuracy: 1.0000
Epoch 487/500
2/2 [==============================] - 0s 5ms/step - loss: 0.0729 - accuracy: 1.0000
Epoch 488/500
2/2 [==============================] - 0s 5ms/step - loss: 0.0720 - accuracy: 1.0000
Epoch 489/500
2/2 [==============================] - 0s 6ms/step - loss: 0.0713 - accuracy: 1.0000
Epoch 490/500
2/2 [==============================] - 0s 5ms/step - loss: 0.0707 - accuracy: 1.0000
Epoch 491/500
2/2 [==============================] - 0s 5ms/step - loss: 0.0701 - accuracy: 1.0000
Epoch 492/500
2/2 [==============================] - 0s 6ms/step - loss: 0.0693 - accuracy: 1.0000
Epoch 493/500
2/2 [==============================] - 0s 5ms/step - loss: 0.0686 - accuracy: 1.0000
Epoch 494/500
2/2 [=====

In [ ]:
# to save the trained model for any future use
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)
    
# Pickle in Python is primarily used in serializing and deserializing a Python object structure. 
# In other words, it's the process of converting a Python object into a byte stream to store it in a file/database, maintain program state across sessions, or transport data over the network.

INFO:tensorflow:Assets written to: chat_model\assets


In [ ]:
import pickle
import numpy as np
from tensorflow.keras.models import load_model

# load the saved tokenizer and label encoder
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

with open('label_encoder.pickle', 'rb') as ecn_file:
    lbl_encoder = pickle.load(ecn_file)

# load the saved model
model = load_model("chat_model")

# define a function to generate responses
def generate_response(user_input):
    # convert user input to a padded sequence of integers using the tokenizer
    sequence = tokenizer.texts_to_sequences([user_input])
    padded_sequence = pad_sequences(sequence, truncating='post', maxlen=20)
    
    # predict the class probabilities for the input sequence using the trained model
    predictions = model.predict(padded_sequence)
    
    # find the index of the class with the highest probability
    class_index = np.argmax(predictions)
    
    # convert the class index back to the corresponding tag using the label encoder
    tag = lbl_encoder.inverse_transform([class_index])[0]
    
    # choose a random response from the list of responses for the predicted tag
    responses = data['intents']
    for intent in responses:
        if intent['tag'] == tag:
            result = np.random.choice(intent['responses'])
            break
            
    return result


In [ ]:
# call the generate_response function with a user input
response = generate_response("Hi there!")

# print the response
print(response)


Hello


In [ ]:
import random
def chat():
    print("Start Talking with the bot(type quit to stop!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = generate_response(inp)
        
        print("Bot: ", results)
        

In [ ]:
chat()

Start Talking with the bot(type quit to stop!
You: hello
Bot:  Hello
You: i need help
Bot:  Yes Sure, How can I support you
You: need
Bot:  Any time!
You: need
Bot:  Any time!


KeyboardInterrupt: Interrupted by user